In [1]:
#importing packages
import os, sys
import numpy as np
import datetime as datetime
from datetime import timedelta
#import matplotlib.pyplot as plt#
from os import getcwd,listdir,chdir
from os.path import isfile, join
import pandas as pd
import filecmp
import string

In [2]:
def _pad_id(idnum):
    return idnum.zfill(6)

def id_treat(id_col):
    id_col = id_col.astype(str)
    id_col = id_col.str.strip()
    id_col = id_col.str.replace(r'(\.0)', '')
    id_col = id_col.replace({'null':np.NaN, 'nan':np.NaN, 'na':np.NaN, '':np.NaN, b'\n':np.NaN, b'\b':np.NaN, b'\t':np.NaN,'':np.NaN})
    id_col.fillna(np.nan, inplace=True)
    return id_col

def salesforce_id_treat(idcol):
    id_treat(idcol)
    idcol = idcol.apply(lambda x: _pad_id(x) if(np.all(pd.notnull(x[1]))) else x)
    return idcol

In [3]:
listofproducers = pd.read_excel('DM-380.xlsx', 'report1525382378173')
categorizedaccounts = pd.read_excel('DM-380.xlsx', 'Cat_ACCTS')
acctswids = pd.read_excel('AccountsWRecordIDs.xlsx')

In [4]:
acctswids['Account Record ID'] = acctswids.apply(lambda row: (row['Account Record ID']), axis=1)

### Setting Accounts w/ Record IDs' Index to Account Name

In [5]:
acctswids = acctswids.set_index('Account Name')

### Joining Categorized Accounts with Accounts with IDs to create new table 'CategorizedWithIDs'

In [6]:
CategorizedWithIDs = categorizedaccounts.join(acctswids, on='Account Name')

### Changing Account Record ID Values from float to int 
* Trying to do it the old way that I know, 
`CategorizedWithIDs['Account Record ID'] = CategorizedWithIDs.apply(lambda row: int(row['Account Record ID']), axis=1)` did not lead to the result I was expecting. There was an error that came up because there is a NaN values that breaks the code because it can't change a NaN to an INT

* So we have to get at it with an apply function that applies the int row function and nests an if statement that does two things: 
    * `numpy.all` will "Test whether all array elements along a given axis evaluate to True", and within that: 
        * `pd.notnull is a "replacement for numpy.isfinite / -numpy.isnan which is suitable for use on object arrays." 
* What we end up receiving is the list of integers that makeup 'Account Record IDs' no longer as floats

In [16]:
CategorizedWithIDs['Account Record ID'] = CategorizedWithIDs.apply(lambda row: str(int(row['Account Record ID'])).zfill(6) if pd.notnull(row['Account Record ID']) else None, axis=1)

ValueError: ("invalid literal for int() with base 10: '37824.0'", 'occurred at index 0')

In [8]:
# listofproducers['Account Record ID'] = listofproducers.apply(lambda row: str(int(row['Account Record ID'])).zfill(6) if pd.notnull(row['Account Record ID']) else None, axis=1)

In [9]:
listofproducers['Account Record ID']= listofproducers['Account Record ID'].astype(str)

In [10]:
listofproducers['Account Record ID']=salesforce_id_treat(listofproducers['Account Record ID'])

In [11]:
CategorizedWithIDs['Account Record ID']= CategorizedWithIDs['Account Record ID'].astype(str)

In [12]:
CategorizedWithIDs['Account Record ID']=salesforce_id_treat(CategorizedWithIDs['Account Record ID'])

In [13]:
#listofproducers= listofproducers.set_index('Account Record ID')

In [14]:
listofproducers

,Account Name,Account Record ID,Unnamed: 2,AKA / DBA Name,Fair Trade ID,FLO ID,Billing Country,Country,Billing City,Certification: Last Modified Date,Certification: Created Date,Certifier,Status,Account Record Type,Status Decision Date,Supply Chain Role,Product Category
0,"Cooperativa Agropecuaria ""Alto Sajama""",000042,112.0,ALTO SAJAMA,1000042.0,850.0,Bolivia,Bolivia,La Paz,2017-08-16,2012-02-21,FLO-Cert,Certified,Supplier,2017-08-15,Exporter; Producer,Coffee
1,Asociación de Pequeños Productores de Café de ...,000047,181.0,APECAFE - Asociación de Pequeños Productores d...,NaN,857.0,El Salvador,El Salvador,San Salvador,2017-01-25,2012-04-24,FLO-Cert,Decertified/Not Certified,Supplier,2017-01-24,Producer,Coffee
2,Bananos Ecologicos de la Linea Noroeste,000053,220.0,BANELINO,NaN,1537.0,DOMINICAN REPUBLIC,Dominican Republic,Mao Valverde,2016-08-16,2012-04-24,FLO-Cert,Certified,Supplier,NaT,Producer,Other
3,Central Local de Cooperativas Agropecuarias de...,000079,827.0,CELCCAR,1000079.0,906.0,BOLIVIA,Bolivia,Carnavi,2016-06-02,2012-02-21,FLO-Cert,Certified,Supplier,NaT,Producer,Coffee
4,Central de Cooperativas Agrarias Cafetaleras -...,000084,2938.0,COCLA - Central de Cooperativas Agrarias Cafet...,1000084.0,820.0,Peru,Peru,NaN,2018-02-12,2012-02-13,FLO-Cert,Certified,Supplier,2018-02-08,Producer,Coffee; Other
5,Central de Cafetaleros del Nor Oriente,000112,2941.0,CECANOR - Central de Cafetaleros del Nor Oriente,1000112.0,885.0,Peru,Peru,Chiclayo,2018-05-03,2012-02-14,FLO-Cert,Certified,Supplier,2018-02-09,Producer,Coffee
6,Central de Cooperativas El Ceibo LTDa,000123,2942.0,El Ceibo - Central de Cooperativas El Ceibo Ltda.,1000123.0,1471.0,Bolivia,Bolivia,La Paz,2016-06-02,2012-02-21,FLO-Cert,Certified,Supplier,NaT,Producer,Cocoa; Coffee
7,Federación de Cooperativas Agrícolas Guatemala,000132,2948.0,FEDECOCAGUA - Federación de Cooperativas Agríc...,1000132.0,891.0,Guatemala,Guatemala,Ciudad de Guatemala,2018-05-03,2012-02-21,FLO-Cert,Certified,Supplier,NaT,Exporter; Producer,Coffee
8,"Indígenas de la Sierra Madre de Motozintla ""Sa...",000164,2956.0,ISMAM - Indígenas de la Sierra Madre de Motozi...,1000164.0,745.0,Mexico,Mexico,Tapachula,2018-05-03,2012-02-21,FLO-Cert,Certified,Supplier,2016-08-24,Producer,Coffee
9,Kilimanjaro Native Cooperative Union,000177,2963.0,KNCU - Kilimanjaro Native Cooperative Union,1000177.0,798.0,Tanzania,United Republic of Tanzania,Moshi,2017-09-25,2012-02-21,FLO-Cert,Decertified/Not Certified,Supplier,2017-09-22,Producer,Coffee


In [15]:
CategorizedWithIDs

,Supplier Acct Groupings,Account Name,Product Family,Supervisor,Account Owner,Last Activity,Last Modified Date,Recently Active?,Account Record ID
0,DEVO,1 1,NaN,NaN,Kiel Stroupe,NaT,2018-02-05,NaN,37824.0
1,PRODUCE,1 800 Flowers,Flowers and Plants,NaN,Midori Pinillos,2012-02-14,2017-08-24,NaN,7359.0
2,UNKNOWN,1000 Friends of Washington,NaN,Billy Linstead Goldsmith,Parker Townley,NaT,2013-08-20,NaN,6308.0
3,UNKNOWN,101 Main Bistro & Wine Bar,NaN,Billy Linstead Goldsmith,Parker Townley,NaT,2017-05-26,NaN,3348.0
4,ADMIN,1500 Broadway Real Estate,NaN,NaN,Michael Carman,2014-06-23,2018-01-01,NaN,10092.0
5,CPG,1893,Composite; Sugar,NaN,Angela Crone,2018-05-01,2018-04-12,NaN,190127.0
6,COFFEE,1951 Coffee,Coffee,Billy Linstead Goldsmith,Parker Townley,2016-02-25,2017-05-26,NaN,316270.0
7,CPG,1INDCRE SA,Cocoa,Gabe Contreras,Kacie Scherler,NaT,2018-04-04,NaN,341738.0
8,UNKNOWN,1ST Class fruit,NaN,Aaron Winters,Default Owner,NaT,2017-05-26,NaN,225808.0
9,COFFEE,2 Sons Vending Incorporated,Coffee,NaN,Maurin Triplett,2014-08-27,2017-05-26,NaN,131705.0


In [509]:
# del CategorizedWithIDs['Account Name']

In [529]:
NEWCrossCheckProducersAccts = pd.merge(CategorizedWithIDs, listofproducers, how='outer', on='Account Record ID')

In [530]:
#CrossCheckProducersAccts = CategorizedWithIDs.join(listofproducers, on='Account Record ID')

In [ ]:
NEWCrossCheckProducersAccts

In [514]:
acctswids

,Account Record ID
Account Name,
1 1,37824
1 800 Flowers,7359
1000 Friends of Washington,6308
101 Main Bistro & Wine Bar,3348
1500 Broadway Real Estate,10092
168 Exports Trading Co,83534
1773 Roasters,52602
1893,190127
18Rabbits,34719


In [515]:
CategorizedWithIDs

,Supplier Acct Groupings,Product Family,Supervisor,Account Owner,Last Activity,Last Modified Date,Recently Active?,Account Record ID
0,DEVO,NaN,NaN,Kiel Stroupe,NaT,2018-02-05,NaN,37824.0
1,PRODUCE,Flowers and Plants,NaN,Midori Pinillos,2012-02-14,2017-08-24,NaN,7359.0
2,UNKNOWN,NaN,Billy Linstead Goldsmith,Parker Townley,NaT,2013-08-20,NaN,6308.0
3,UNKNOWN,NaN,Billy Linstead Goldsmith,Parker Townley,NaT,2017-05-26,NaN,3348.0
4,ADMIN,NaN,NaN,Michael Carman,2014-06-23,2018-01-01,NaN,10092.0
5,CPG,Composite; Sugar,NaN,Angela Crone,2018-05-01,2018-04-12,NaN,190127.0
6,COFFEE,Coffee,Billy Linstead Goldsmith,Parker Townley,2016-02-25,2017-05-26,NaN,316270.0
7,CPG,Cocoa,Gabe Contreras,Kacie Scherler,NaT,2018-04-04,NaN,341738.0
8,UNKNOWN,NaN,Aaron Winters,Default Owner,NaT,2017-05-26,NaN,225808.0
9,COFFEE,Coffee,NaN,Maurin Triplett,2014-08-27,2017-05-26,NaN,131705.0


In [516]:
listofproducers

,Account Name,Account Record ID,Unnamed: 2,AKA / DBA Name,Fair Trade ID,FLO ID,Billing Country,Country,Billing City,Certification: Last Modified Date,Certification: Created Date,Certifier,Status,Account Record Type,Status Decision Date,Supply Chain Role,Product Category
0,"Cooperativa Agropecuaria ""Alto Sajama""",000042,112.0,ALTO SAJAMA,1000042.0,850.0,Bolivia,Bolivia,La Paz,2017-08-16,2012-02-21,FLO-Cert,Certified,Supplier,2017-08-15,Exporter; Producer,Coffee
1,Asociación de Pequeños Productores de Café de ...,000047,181.0,APECAFE - Asociación de Pequeños Productores d...,NaN,857.0,El Salvador,El Salvador,San Salvador,2017-01-25,2012-04-24,FLO-Cert,Decertified/Not Certified,Supplier,2017-01-24,Producer,Coffee
2,Bananos Ecologicos de la Linea Noroeste,000053,220.0,BANELINO,NaN,1537.0,DOMINICAN REPUBLIC,Dominican Republic,Mao Valverde,2016-08-16,2012-04-24,FLO-Cert,Certified,Supplier,NaT,Producer,Other
3,Central Local de Cooperativas Agropecuarias de...,000079,827.0,CELCCAR,1000079.0,906.0,BOLIVIA,Bolivia,Carnavi,2016-06-02,2012-02-21,FLO-Cert,Certified,Supplier,NaT,Producer,Coffee
4,Central de Cooperativas Agrarias Cafetaleras -...,000084,2938.0,COCLA - Central de Cooperativas Agrarias Cafet...,1000084.0,820.0,Peru,Peru,NaN,2018-02-12,2012-02-13,FLO-Cert,Certified,Supplier,2018-02-08,Producer,Coffee; Other
5,Central de Cafetaleros del Nor Oriente,000112,2941.0,CECANOR - Central de Cafetaleros del Nor Oriente,1000112.0,885.0,Peru,Peru,Chiclayo,2018-05-03,2012-02-14,FLO-Cert,Certified,Supplier,2018-02-09,Producer,Coffee
6,Central de Cooperativas El Ceibo LTDa,000123,2942.0,El Ceibo - Central de Cooperativas El Ceibo Ltda.,1000123.0,1471.0,Bolivia,Bolivia,La Paz,2016-06-02,2012-02-21,FLO-Cert,Certified,Supplier,NaT,Producer,Cocoa; Coffee
7,Federación de Cooperativas Agrícolas Guatemala,000132,2948.0,FEDECOCAGUA - Federación de Cooperativas Agríc...,1000132.0,891.0,Guatemala,Guatemala,Ciudad de Guatemala,2018-05-03,2012-02-21,FLO-Cert,Certified,Supplier,NaT,Exporter; Producer,Coffee
8,"Indígenas de la Sierra Madre de Motozintla ""Sa...",000164,2956.0,ISMAM - Indígenas de la Sierra Madre de Motozi...,1000164.0,745.0,Mexico,Mexico,Tapachula,2018-05-03,2012-02-21,FLO-Cert,Certified,Supplier,2016-08-24,Producer,Coffee
9,Kilimanjaro Native Cooperative Union,000177,2963.0,KNCU - Kilimanjaro Native Cooperative Union,1000177.0,798.0,Tanzania,United Republic of Tanzania,Moshi,2017-09-25,2012-02-21,FLO-Cert,Decertified/Not Certified,Supplier,2017-09-22,Producer,Coffee


In [517]:
categorizedaccounts

,Supplier Acct Groupings,Account Name,Product Family,Supervisor,Account Owner,Last Activity,Last Modified Date,Recently Active?
0,DEVO,1 1,NaN,NaN,Kiel Stroupe,NaT,2018-02-05,NaN
1,PRODUCE,1 800 Flowers,Flowers and Plants,NaN,Midori Pinillos,2012-02-14,2017-08-24,NaN
2,UNKNOWN,1000 Friends of Washington,NaN,Billy Linstead Goldsmith,Parker Townley,NaT,2013-08-20,NaN
3,UNKNOWN,101 Main Bistro & Wine Bar,NaN,Billy Linstead Goldsmith,Parker Townley,NaT,2017-05-26,NaN
4,ADMIN,1500 Broadway Real Estate,NaN,NaN,Michael Carman,2014-06-23,2018-01-01,NaN
5,CPG,1893,Composite; Sugar,NaN,Angela Crone,2018-05-01,2018-04-12,NaN
6,COFFEE,1951 Coffee,Coffee,Billy Linstead Goldsmith,Parker Townley,2016-02-25,2017-05-26,NaN
7,CPG,1INDCRE SA,Cocoa,Gabe Contreras,Kacie Scherler,NaT,2018-04-04,NaN
8,UNKNOWN,1ST Class fruit,NaN,Aaron Winters,Default Owner,NaT,2017-05-26,NaN
9,COFFEE,2 Sons Vending Incorporated,Coffee,NaN,Maurin Triplett,2014-08-27,2017-05-26,NaN


In [518]:
Output = CrossCheckProducersAccts[CrossCheckProducersAccts['Account Name'].notnull()]

In [519]:
pd.notnull(CrossCheckProducersAccts['Account Name']).unique()

array([False,  True])

In [520]:
CrossCheckProducersAccts.to_csv('DM-380.csv',index=False, encoding='utf-8')

In [521]:
Output

,Supplier Acct Groupings,Product Family,Supervisor,Account Owner,Last Activity,Last Modified Date,Recently Active?,Account Record ID,Account Name,Unnamed: 2,...,Billing City,Created Date,Certification: Last Modified Date,Certification: Created Date,Certifier,Status,Account Record Type,Status Decision Date,Supply Chain Role,Product Category
97,COFFEE,Coffee,Paul Alvarez,Angela Reed,2017-05-19,2018-04-13,NaN,002910,Abahuzamugambi Ba Kawa Ba Maraba,NaN,...,Butare,2006-07-20,2016-06-02,2012-02-21,FLO-Cert,Certified,Supplier,NaT,Producer,Coffee
98,COFFEE,Coffee,Paul Alvarez,Angela Reed,2011-05-12,2018-01-17,NaN,002911,Abakunda Kawa,NaN,...,NaN,2006-07-20,2016-06-02,2012-04-24,FLO-Cert,Dormant,Supplier,NaT,Producer,Coffee
99,COFFEE,Coffee,Paul Alvarez,Angela Reed,NaT,2018-01-17,NaN,009744,Abateganda Ntungamo GCS,NaN,...,Ntungamo,2008-07-29,2016-06-02,2012-04-24,FLO-Cert,Certified,Supplier,NaT,Producer,Coffee
162,AHG,Apparel and Home Goods,Eva Pan,Stephanie Doe,2018-05-01,2018-05-03,NaN,347508,Accurate Weave PVT LTD (Unit - 1),NaN,...,Mirzapur,2017-03-23,2018-05-03,2017-03-23,FTUSA,Certified,Supplier,2018-04-20,Factory,Apparel
188,COFFEE,Coffee,Paul Alvarez,Angela Reed,2009-04-03,2018-01-17,NaN,002914,ACU,NaN,...,Arusha,2006-07-20,2016-06-02,2012-04-24,FLO-Cert,NaN,Supplier,NaT,Producer,Coffee
236,COFFEE,Coffee,Paul Alvarez,Angela Reed,NaT,2018-01-17,NaN,002915,ADERSO,NaN,...,NaN,2006-07-20,2016-06-02,2012-04-24,FLO-Cert,NaN,Supplier,NaT,Producer,Coffee
259,PRODUCE,Produce,Paul Alvarez,Angela Reed,2010-10-27,2018-01-17,NaN,019438,Adonten Pineapple Grower & Marketing Society,NaN,...,Nsawam,2010-03-08,2016-08-16,2012-04-24,FLO-Cert,Certified,Supplier,NaT,Producer,Pineapples
288,AHG,Apparel and Home Goods,Eva Pan,Stephanie Doe,2018-05-03,2018-05-03,NaN,171107,Aesthetic Living Merchants (Pvt) LTD,NaN,...,Pace City-2,2015-03-03,2018-05-03,2015-04-08,FTUSA,Certified,Supplier,2015-10-02,Factory,Home Goods
293,COFFEE,Coffee,Paul Alvarez,Angela Reed,2009-04-03,2018-01-17,NaN,002919,Africa Coffee Company,NaN,...,Moshi,2006-07-20,2016-06-02,2012-04-24,FLO-Cert,NaN,Supplier,NaT,Producer,Coffee
324,AHG,Apparel and Home Goods,Paul Alvarez,Angela Reed,NaT,2018-01-17,NaN,002922,Agri - Industrial Management Agency LTD,NaN,...,NaN,2006-07-20,2016-06-02,2012-04-24,FLO-Cert,NaN,Supplier,NaT,Producer,NaN


In [522]:
Output.to_csv('Output.csv',index = False,encoding='latin1')

In [523]:
NEWCrossCheckProducersAccts.to_csv('NEWCrossCheckProducersAccts.csv',index = False,encoding='latin1')